# Setup

In [1]:
from openai import OpenAI
from openai import embeddings

In [2]:
from dotenv import load_dotenv
import os
import json

load_dotenv(dotenv_path=".openaidev.env")



True

In [3]:
print(os.getenv("OLLAMA_BASE_URL")[:20], "...")


http://localhost:114 ...


In [4]:
client = OpenAI(
    base_url = os.getenv("OLLAMA_BASE_URL"),
    api_key='ollama', # required, but unused
)


In [ ]:

response = client.chat.completions.create(
  model="gpt-oss",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The LA Dodgers won in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)
print(response.choices[0].message.content)

In [ ]:
print(response.to_json())


# Embeddings



In [ ]:
embedding_model = "nomic-embed-text"

In [ ]:
embedding = client.embeddings.create(
    model=embedding_model,
    input=["Hello, world!", "Hola!"]
)

In [ ]:
[type(e) for e in embedding]

In [ ]:
for e in embedding:
    print(e)

In [ ]:
embedding.data[0].embedding[:20]

In [ ]:
embedding.data[1].embedding[:20]

## Cosine similarity

In [5]:
import numpy as np

In [ ]:
# input=["feline friends say", "meow"]
input=['Hello, world!', 'Hola!']

resp = client.embeddings.create(
    model=embedding_model,
    input=input,

)

embedding_a = resp.data[0].embedding
embedding_b = resp.data[1].embedding

similarity_score = np.dot(embedding_a, embedding_b)
print(similarity_score)


In [ ]:
input=[
    'Azure OpenAI is a cloud-based AI platform.',
    'The Eiffel Tower.'
    ]

resp = client.embeddings.create(
    model=embedding_model,
    input=input,

)

embedding_a = resp.data[0].embedding
embedding_b = resp.data[1].embedding

similarity_score = np.dot(embedding_a, embedding_b)
print(similarity_score)

In [ ]:
input=[
    'The Sydney Harbour Bridge.',
    'The Eiffel Tower.'
    ]

resp = client.embeddings.create(
    model=embedding_model,
    input=input,

)

embedding_a = resp.data[0].embedding
embedding_b = resp.data[1].embedding

similarity_score = np.dot(embedding_a, embedding_b)
print(similarity_score)

# Responses API for gpt-oss


- use responsed api for gpt-oss
- use structured output


Ollama doesn’t (yet) support the Responses API natively.
https://cookbook.openai.com/articles/gpt-oss/run-locally-ollama

In [6]:
from pydantic import BaseModel, Field
from typing import List
class ItemPrice(BaseModel):
    item: str
    price: float

class ItemPriceResponse(BaseModel):
    items: List[ItemPrice]

user_input = "Extract product names and prices: apples, $4, bananas, $3, cherries, $5"


In [7]:


structured_output_response = client.chat.completions.create(
    model="gpt-oss",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_input},
        # {"role": "user", "content":
        # "List 3 items with prices in a JSON array, with fields item and price."}
    ],
    # response_format=ItemPriceResponse
)


In [8]:
print(structured_output_response.choices[0].message.content)

**Product names and prices**

| Product  | Price |
|----------|-------|
| Apples   | $4    |
| Bananas  | $3    |
| Cherries | $5    |

*You can also use the following simple list format:*

- **Apples** – $4  
- **Bananas** – $3  
- **Cherries** – $5

Feel free to let me know if you’d like them in a different format (e.g., JSON, CSV, etc.)!


In [9]:
structured_output_response = client.chat.completions.parse(
    model="llama3.2",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_input},
        # {"role": "user", "content": "List 3 items with prices in a JSON array, with fields item and price."}
    ],
    response_format=ItemPriceResponse
)
print(structured_output_response.choices[0])

ParsedChoice[ItemPriceResponse](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[ItemPriceResponse](content='{\n  "items": [\n    {\n      "item": "apples",\n      "price": 4\n    },\n    {\n      "item": "bananas",\n      "price": 3\n    },\n    {\n      "item": "cherries",\n      "price": 5\n    }\n  ]\n}', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, parsed=ItemPriceResponse(items=[ItemPrice(item='apples', price=4.0), ItemPrice(item='bananas', price=3.0), ItemPrice(item='cherries', price=5.0)])))


In [10]:
print(structured_output_response.choices[0].message)

ParsedChatCompletionMessage[ItemPriceResponse](content='{\n  "items": [\n    {\n      "item": "apples",\n      "price": 4\n    },\n    {\n      "item": "bananas",\n      "price": 3\n    },\n    {\n      "item": "cherries",\n      "price": 5\n    }\n  ]\n}', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, parsed=ItemPriceResponse(items=[ItemPrice(item='apples', price=4.0), ItemPrice(item='bananas', price=3.0), ItemPrice(item='cherries', price=5.0)]))


In [11]:
choice_0 = structured_output_response.choices[0].message.parsed
print(type(choice_0))
for item in choice_0.items:
    print("item name: ", item.item, ", price: ", item.price)

<class '__main__.ItemPriceResponse'>
item name:  apples , price:  4.0
item name:  bananas , price:  3.0
item name:  cherries , price:  5.0


In [ ]:
type(choice_0)

In [ ]:
print(choice_0)

In [ ]:
choice_0.content

In [ ]:
structured_output_response

In [ ]:
print(structured_output_response)

# Images

In [13]:
# images


input=[
    {"role": "assistant", "content": "You are a helpful assistant."},
    {
        "role": "user",
        "content": [

                { "type": "input_text", "text": "describe " },
                {
                    "type": "input_image",
                    "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
                }
        ]
    }
]



output = client.chat.completions.create(
    model="gpt-oss",
    messages=[input],
    # response_format=ItemPriceResponse
)


BadRequestError: Error code: 400 - {'error': {'message': 'json: cannot unmarshal array into Go struct field ChatCompletionRequest.messages of type openai.Message', 'type': 'invalid_request_error', 'param': None, 'code': None}}